In [9]:
import os
import re
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import pandas as pd
import pickle

In [2]:
def save_pickle(obj, fname):
    try:os.mkdir('obj/')
    except:pass
    with open('obj/'+fname + '.pk', 'wb+') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
def load_pickle(fname):
    with open('obj/' + fname + '.pk', 'rb') as f:
        return pickle.load(f)

In [4]:
def write_csv(bag,brand,additional_info='_'):
    import csv
    import random
    import pandas as pd
    try:os.mkdir('comments_AMAZON/')
    except:pass
    df = pd.DataFrame.from_dict(bag, orient='index').T.to_csv('comments_AMAZON/'+brand+str(random.randint(1,1000000))+additional_info+'.csv',index=False)
    print('CSV IS WRITTEN')

In [5]:
#Find Stars
def star_finder(driver,STARS_LIST):
    for STAR in driver.find_elements_by_css_selector('.a-link-normal'):
        a = re.search('(.+) out.*',STAR.get_attribute('title'))
        if a:
            STARS_LIST.append(a.group()[0])
        else:pass
    return STARS_LIST

In [6]:
def get_phone_urls_prices(driver,brand):
    url_list=list()
    name_price = dict()
    # How many catalog pages are there?
    try:catalog_pages = int(driver.find_element_by_class_name('pagnDisabled').text)
    except:
        try:catalog_pages = int(driver.find_elements_by_class_name('pagnLink')[-1].text)
        except:catalog_pages = 2

    # Move onto next catalog page.
    for page_nu in range(1,catalog_pages):
        # Next Catalog Page
        # If first page is done, continue to other pages
        
        # GET PRICES
        print("# Finding Phone Names and Prices in Catalog Page...")
        html = driver.page_source
        soup = BeautifulSoup(html, "lxml")
        dirtylist = soup.find('li',{'id':re.compile('result_\d')}).find_all_next(string=True)
        biglist = list(dirtylist)
        phones = [i for i, e in enumerate(biglist) if e == brand]
        for n in phones:
            phonename = biglist[n-2]
            phoneprice = biglist[n+1]
            name_price[phonename]=phoneprice
            print("phonename: {},\nPhoneprice {}".format(phonename, phoneprice))

        # GET URLS
        for element in driver.find_elements_by_css_selector('.a-link-normal.s-access-detail-page.s-color-twister-title-link.a-text-normal'):
            url_list.append(element.get_attribute('href'))
            print(element.get_attribute('href')) #pRint url
        
        # Next Page
        try:
            next_url = driver.find_element_by_id('pagnNextLink').get_attribute('href')
            driver.get(next_url)
            print("# Moving on to next catalog page...")
            driver.find_element_by_tag_name('html').send_keys(Keys.END)
            time.sleep(10)
        except: print('There is no other phone page.')

    print("Url gathering process has finished.")
    return url_list,name_price


In [10]:
#path_to_extension = r"C:\Users\dogus\Dropbox\DgsPy_DBOX\webdrivers\1.16.14_0"
path = "C:/Users/dogus/Dropbox/DgsPy_DBOX/webdrivers/chromedriver.exe"

#chrome_options = Options()
#chrome_options.add_argument('load-extension=' + path_to_extension)
#chrome_options.add_argument("--headless")

driver = webdriver.Chrome(executable_path=path)#,chrome_options=chrome_options)
#driver.create_options()

main_url ="https://www.amazon.com/gp/search/other/ref=lp_7072561011_sa_p_89?rh=n%3A2335752011%2Cn%3A%212335753011%2Cn%3A7072561011&bbn=7072561011&pickerToList=lbr_brands_browse-bin&ie=UTF8&qid=1532365672"
driver.get(main_url)

wait = WebDriverWait(driver,10)

In [7]:
driver.quit()

brand_list = ['Alcatel','Alcatel One Touch','Apple','Asus','BlackBerry','Blackview','BLU','BlueCosmo','Casio','CAT PHONES','CNPGD','CUBOT',
'DOOGEE', 'Easyfone', 'FreedomPop', 'Google','GreatCall','H2O Wireless','Honor','HTC','Huawei', 'Jethro','Kyocera','Leeco','LeEco', 'LG', 
'Microsoft','mophie','Motorola','Newmind','Nokia','NUU','OnePlus','Pantech','Plum', 'Razer', 'RCA','Samsung','Seneo','Simple Mobile', 
'Sony','Tile','Total Wireless','Unihertz','Xiaomi', 'ZTE', 'ZTE USA']

In [11]:
def main():
    brand_list = [
        #'BLU',
        #'Blackview',
        #'Sony',
        #'Asus',
        #'Google',
        #'Xiaomi',
        #'HTC',
        #'Huawei'
        'Samsung'
        #'LG'
                 ]
    
    for brand in brand_list:
        xpath = '//a[@title="{}"]'.format(brand)
    
        # Move on to the MAIN catalog
        driver.find_element_by_xpath(xpath).click()
        if False:
            # Find Phone Names and Prices in Catalog Page
            urls,name_price =get_phone_urls_prices(driver,brand=brand) ######
            phone_urls = urls
            #write_csv(name_price,brand,additional_info='Prices')
            save_pickle(phone_urls,brand)
        else:
            phone_urls = load_pickle(brand)
            
        # Iterate over phones on pages
        print("# Iterate over phones on pages")
        url_amount = len(phone_urls)
        for phone_url in phone_urls[252:]:
            print("Amount of urls have left {}".format(url_amount))
            print('Working on the next phone.\n', phone_url)
            phone_code = phone_url.split('/')[5]
            # Go to that specific phone's comment page.
            link = 'https://www.amazon.com/product-reviews/'+phone_code+'/ref=cm_cr_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1'
            time.sleep(20) # Sleep
            driver.get(link)
            try:last_page_number = driver.find_elements_by_class_name('page-button')[-1].text
            except:
                last_page_number = 1
                print('There is no comment page.')

            # Iteration on Comment Pages
            for c_page_nu in range(1,int(last_page_number)+1):
                text_list=list()
                phone_title = list()
                # Get down below, comments gets loaded on page
                driver.find_element_by_tag_name('html').send_keys(Keys.END)
                time.sleep(15)
                # If first page is done, continue to other page
                nextbutton=True
                if c_page_nu!=1:
                    try:
                        wait.until(EC.visibility_of_element_located((By.XPATH,'//li[@class="a-last"]')))
                        next_button = driver.find_element_by_xpath('//li[@class="a-last"]')
                        next_button.click()
                        time.sleep(5)
                        print('Moving on to the page %s' % c_page_nu)
                    except:
                        print('No "Next Page" button')
                        nextbutton=False
                if nextbutton==True:
                    driver.find_element_by_tag_name('html').send_keys(Keys.END)
                    time.sleep(10)
                    # Comment Stars
                    stars_list = list()
                    try:wait.until(EC.visibility_of_element_located((By.CLASS_NAME,'a-link-normal')))
                    except:print('WARNING:There was no comment stars to be seen!') 
                    for STAR in driver.find_elements_by_class_name('a-link-normal'):
                        #print(STAR)
                        star_line = STAR.get_attribute('title')
                        a = re.search('(.+) out.*',star_line)
                        if a:stars_list.append(a.group()[0])
                        else:pass

                    #  TEXT: Name, Comment, Comment Title, Date, Verified_Purchase
                    text_list=list()
                    phone_title = list()
                    try:wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.a-section.review')))
                    except:print('WARNING:There was no comments to be seen!') 
                    COMMENT_ELEMENT = driver.find_elements_by_css_selector('.a-section.review')
                    for person in COMMENT_ELEMENT:
                        text = person.text
                        text_list.append(text)

                        # Add Phone Titles
                        phone_title.append(driver.find_elements_by_class_name('a-link-normal')[0].text)
                        # Add Phone Prices
                        #phone_price.append(price)
                    print('Text data has been gathered.')
            # Iteration over one phone has finished, write down to a csv file.
            url_amount -= 1
            if len(text_list)!=0:
                big_dict = {
                    'Text':text_list,
                    'Phone Title':phone_title,
                    'Stars':stars_list,
                    }
                write_csv(big_dict,brand)
            else:print('WARNING: There was nothing in the text_list.')

        print('Number of {} phones iterated: {}'.format(brand,len(phone_urls)))
        #print('Amout of comments have been gathered for the {}: {}'.format(brand,len(text_list)))
        time.sleep(15) # Sleep
        driver.get(main_url)
        # Write the dictionaries into csv files.
if __name__=='__main__':
    main()


# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung Galaxy S8 Unlocked 64GB - US Version (Midnight Black) - US Warranty,
Phoneprice $499.99
phonename: Samsung Galaxy J7 Prime (32GB) G610F/DS - 5.5" Dual SIM Unlocked Phone with Finger Print Sensor (Gold),
Phoneprice $203.40
phonename: Samsung Galaxy S9 Unlocked Smartphone - Midnight Black - US Warranty,
Phoneprice $683.95
phonename: Samsung Galaxy J7 Neo (16GB) J701M/DS - 5.5", Android 7.0, Dual SIM Unlocked Smartphone, International Model - Gold,
Phoneprice $159.00
phonename: Samsung Galaxy S7 G930v 32GB Verizon Wireless CDMA 4G LTE Smartphone w/12MP Camera - Black Onyx (Certified Refurbished),
Phoneprice $248.95
phonename: Samsung Galaxy J2 Prime (16GB) 5.0" 4G LTE GSM Dual SIM Factory Unlocked International Version, No Warranty G532M/DS (Black),
Phoneprice $108.50
phonename: Samsung Galaxy S7 SM-G930A AT&T Unlocked Smartphone, (Black Onyx),
Phoneprice $300.95
phonename: Samsung Galaxy J7 Pro (32GB) J730G/DS - 5.5" 

# Moving on to next catalog page...
# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung Galaxy G550T On5 GSM Unlocked 4G LTE Android Smartphone - Black - (NOT FOR Metro PCS) (Certified Refurbished),
Phoneprice $74.99
phonename: Samsung Galaxy S8+ 64GB Unlocked Phone - 6.2" Screen - International Version (Midnight Black),
Phoneprice $520.00
phonename: Samsung Galaxy S9+ Unlocked Smartphone - Midnight Black - US Warranty (Certified Refurbished),
Phoneprice $710.99
phonename: Samsung Galaxy J3 Pro 2017 (16GB) J330G/DS - 5.0" Dual SIM Factory Unlocked Phone (US & Latin 4G LTE) (Gold),
Phoneprice $199.00
phonename: Samsung Galaxy S5 G900A 16 GB 4G LTE (Shimmery White) GSM Unlocked,
Phoneprice $170.00
phonename: Samsung S7 Unlocked GSM Smartphone, Gold, 32GB,
Phoneprice $290.95
phonename: Samsung Galaxy S7 Edge 32GB G935A GSM Unlocked (Certified Refurbished) (Black),
Phoneprice $327.95
phonename: Samsung Galaxy Note 8 SM-N950F/DS Factory Unlocked Phone - 6.3" Screen - 64GB

# Moving on to next catalog page...
# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung S7 EDGE G935V 32GB, Verizon/GSM Unlocked, Silver Titanium (Certified Refurbished),
Phoneprice $309.99
phonename: Samsung Galaxy S9 Plus (SM-G965F/DS) 6GB/128GB 6.2-inches LTE Dual SIM Factory Unlocked - International Stock No Warranty (Midnight Black),
Phoneprice $769.17
phonename: Samsung Galaxy S7 G930T 32GB T-Mobile - Black,
Phoneprice $158.93
phonename: Samsung Galaxy S7 G930A 32GB AT&T GSM 4G LTE Smartphone - Gold Platinum,
Phoneprice $293.99
phonename: Samsung Galaxy Note 3 N900 32GB Unlocked GSM 4G LTE Android Smartphone w/S Pen Stylus - Black,
Phoneprice $249.00
phonename: Samsung Galaxy Grand Prime GSM Unlocked Cellphone - Gray,
Phoneprice $149.99
phonename: Samsung Galaxy J2 Prime G532M 16GB Unlocked GSM 4G LTE Quad-Core Phone w/8MP Camera - Black,
Phoneprice $111.70
phonename: Samsung Galaxy A8+ (2018) Factory Unlocked SM-A730F/DS DUAL SIM 64GB/4GB Ram, 6" Screen, 16MP 

# Moving on to next catalog page...
# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung Galaxy S9 Plus (SM-G9650/DS) 6GB / 128GB 6.2-inches LTE Dual SIM Factory Unlocked - International Stock No Warranty (Midnight Black),
Phoneprice $774.00
phonename: Samsung Galaxy Mega 6.3 I527 16GB Unlocked GSM 4G LTE Smartphone w/8MP Camera - Black,
Phoneprice $212.92
phonename: Samsung Galaxy S5 G900V Verizon 4G LTE Smartphone w/ 16MP Camera - Black - Verizon,
Phoneprice $175.50
phonename: Samsung Galaxy J3 (2016) J320V Verizon CDMA 4G LTE Quad-Core Phone w/8MP Camera- Black,
Phoneprice $119.99
phonename: Samsung Galaxy Note 4 N910a 32GB GSM Unlocked Smartphone Charcoal Black (Certified Refurbished),
Phoneprice $249.99
phonename: Samsung Stratosphere I405 4G LTE Verizon CDMA Android Slider Cell Phone - Black,
Phoneprice $22.95
phonename: Samsung Galaxy S4 Active I537 16GB Unlocked GSM 4G LTE Water-Resistant Smartphone w/ 13MP Camera - Urban Gray,
Phoneprice $155.00
phonename: Sa

# Moving on to next catalog page...
# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung Galaxy Rugby Pro, Black 8GB (AT&T),
Phoneprice $101.14
phonename: Samsung Galaxy J4 J400M/DS 16GB 2GB RAM 5.5" 4G LTE GSM Dual SIM Factory Unlocked International Version, No Warranty (Gold),
Phoneprice $140.00
phonename: Samsung Galaxy S7 G930A Unlocked Smartphone, (Black Onyx),
Phoneprice $299.95
phonename: Samsung galaxy Note 8 SM-N950U 64GB Orchid Gray (Verizon Wireless) (Certified Refurbished),
Phoneprice $584.99
phonename: Samsung Galaxy S5 Charcoal Black - No Contract Phone (U.S. Cellular),
Phoneprice $94.38
phonename: Samsung Galaxy Note 5 Verizon Wireless 4G LTE Smartphone with Stylus Pen - Black Sapphire,
Phoneprice $187.49
phonename: Samsung Galaxy S7 Edge SM-G935A 32GB Smartphone for AT&T (Certified Refurbished),
Phoneprice $324.00
phonename: Samsung Galaxy S6 Edge, White Pearl 64GB (Verizon Wireless),
Phoneprice $239.77
phonename: Samsung Galaxy Mega 2 G750a Unlocked G

...

# Moving on to next catalog page...
# Finding Phone Names and Prices in Catalog Page...
phonename: Samsung Galaxy S8+ Plus SM-G955FD 64GB Dual Sim Unlocked Phone -US/Latin America Version (Maple Gold),
Phoneprice $679.95
phonename: Samsung Galaxy S8+ (64GB) G955U 6.2" 4G LTE Unlocked (GSM + Verizon, US WARRANTY) (Midnight Black),
Phoneprice $409.99
phonename: Samsung SM-G950FD Unlocked Smartphone, Galaxy S8 Duos 64GB, 1-Year B&H Warranty - International Variant/US Compatible LTE (Midnight Black),
Phoneprice $476.00
phonename: Samsung G925T S6 Edge Unlocked S&D, Gold, 32 GB (Refurbished),
Phoneprice $425.00
phonename: Samsung Galaxy S6 Edge (Verizon) Certified Pre-Owned Prepaid Carrier Locked, Gold,
Phoneprice $350.59
phonename: Straight Talk Samsung Galaxy S6 "White Pearl" 32GB runs on Verizon's 4G XLTE Via Straight Talk's $45.00 5GB Unlimited talk & Text "Service Card Not included",
Phoneprice $467.00
phonename: Samsung S7 Edge Silver Titanium + Samsung Gear VR *Both Brand New* (T-Mob

Moving on to the page 3
Text data has been gathered.
Moving on to the page 4
Text data has been gathered.
Moving on to the page 5
Text data has been gathered.
Moving on to the page 6
Text data has been gathered.
Moving on to the page 7
Text data has been gathered.
Moving on to the page 8
Text data has been gathered.
Moving on to the page 9
Text data has been gathered.
Moving on to the page 10
Text data has been gathered.
Moving on to the page 11
Text data has been gathered.
Moving on to the page 12
Text data has been gathered.
Moving on to the page 13
Text data has been gathered.
Moving on to the page 14
Text data has been gathered.
Moving on to the page 15
Text data has been gathered.
Moving on to the page 16
Text data has been gathered.
Moving on to the page 17
Text data has been gathered.
Moving on to the page 18
Text data has been gathered.
Moving on to the page 19
Text data has been gathered.
Moving on to the page 20
Text data has been gathered.
Moving on to the page 21
Text data 

Text data has been gathered.
Moving on to the page 2
Text data has been gathered.
CSV IS WRITTEN
Amount of urls have left 1383
Working on the next phone.
 https://www.amazon.com/Samsung-Galaxy-Prime-G360T-T-Mobile/dp/B0176Z54AC/ref=sr_1_224?s=wireless&ie=UTF8&qid=1533380121&sr=1-224&refinements=p_89%3ASamsung
Text data has been gathered.
Moving on to the page 2
Text data has been gathered.
Moving on to the page 3
Text data has been gathered.
Moving on to the page 4
Text data has been gathered.
Moving on to the page 5
Text data has been gathered.
Moving on to the page 6
Text data has been gathered.
Moving on to the page 7
Text data has been gathered.
Moving on to the page 8
Text data has been gathered.
Moving on to the page 9
Text data has been gathered.
Moving on to the page 10
Text data has been gathered.
CSV IS WRITTEN
Amount of urls have left 1382
Working on the next phone.
 https://www.amazon.com/Samsung-Verizon-Unlocked-Certified-Refurbished/dp/B079YZ8J7Z/ref=sr_1_225?s=wireless&i

There is no comment page.
Text data has been gathered.
CSV IS WRITTEN
Amount of urls have left 1363
Working on the next phone.
 https://www.amazon.com/Samsung-GT-i8190-Unlocked-International-Version/dp/B00A29WCA0/ref=sr_1_244?s=wireless&ie=UTF8&qid=1533380133&sr=1-244&refinements=p_89%3ASamsung
There is no comment page.
Text data has been gathered.
Amount of urls have left 1362
Working on the next phone.
 https://www.amazon.com/Samsung-Unlocked-Fingerprint-International-Warranty/dp/B07DJST8L9/ref=sr_1_245?s=wireless&ie=UTF8&qid=1533380133&sr=1-245&refinements=p_89%3ASamsung
There is no comment page.
Text data has been gathered.
Amount of urls have left 1361
Working on the next phone.
 https://www.amazon.com/Samsung-Charge-Android-Smartphone-Verizon/dp/B006YCU5MQ/ref=sr_1_246?s=wireless&ie=UTF8&qid=1533380133&sr=1-246&refinements=p_89%3ASamsung
There is no comment page.
Text data has been gathered.
Amount of urls have left 1360
Working on the next phone.
 https://www.amazon.com/Samsung-

WebDriverException: Message: chrome not reachable
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.17134 x86_64)


## Methods & Test Area

In [ ]:
# page 103,
#BLU R1 HD Cell Phone 16GB - Black ›
#B01H2E0KVA
" asd "+None+"ggg"

In [ ]:
wait = WebDriverWait(driver,10)
try:
    wait.until(EC.visibility_of_element_located((By.ID,"pagnNextLink")))
    next_button = driver.find_element_by_id('pagnNextLink')
    next_button.click()
except:
    print('No "Next Page" button..')

In [ ]:
wait = WebDriverWait(driver,10)
try:
    wait.until(EC.visibility_of_element_located((By.XPATH,'//li[@class="a-last"]')))
    next_button = driver.find_element_by_xpath('//li[@class="a-last"]')
    next_button.click()
except:
    print('No "Next Page" button..')

In [ ]:
driver.find_elements_by_css_selector('.a-section.review')[5].text

In [ ]:
len(text_list)

In [ ]:
# Some Techniques:
for e in driver.find_elements_by_xpath("//*[contains(text(), 'stars') and @class='a-icon-alt']"):
    print(e.text)
#'//div[contains(text(), "{0}") and @class="inner"]'.format(text)
#driver.find_elements_by_xpath("//div[contains(text(), '$')]")

In [ ]:
driver.find_element_by_css_selector('.sx-price.sx-price-large').get_attribute('innerHTML')

In [ ]:
# next, getNext()
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
soup.find('li',{'id':re.compile('result_\d')}).next.getText()

In [ ]:
# Find Phone Names and Prices in Catalog Page
html = driver.page_source
soup = BeautifulSoup(html, "lxml")

dirtylist = soup.find('li',{'id':re.compile('result_\d')}).find_all_next(string=True)
biglist = list(dirtylist)

name_price = dict()
apples = [i for i, e in enumerate(biglist) if e == 'H2O Wireless']
for n in apples:
    phonename = biglist[n-2]
    phoneprice = biglist[n+1]
    name_price[phonename]=phoneprice
    print("phonename: {},\nPhoneprice {}".format(phonename, phoneprice))

In [ ]:
#Find Stars
html = driver.page_source
soup = BeautifulSoup(html, "lxml")
dirtylist = soup.find('span',{'class':'a-icon-alt'})#.getText()#re.compile('result_\d')}).find_all_next(string=True)
for star_line in dirtylist.find_all_next(string=True):
    a = re.search('(\d\W\d) out of 5 stars',star_line)
    if a:
        stars_list.append(a.group()[0])
    else:pass

In [ ]:
next_url = URL.replace("sr_pg_{}".format(page_nu-1),"sr_pg_{}".format(page_nu)).replace("page={}".format(page_nu-1),"page={}".format(page_nu))
print(next_url)
driver.get(next_url)
print("# Moving on to next catalog page...")
driver.find_element_by_tag_name('html').send_keys(Keys.END)
time.sleep(10)